# 02 - ReAct Research Paper Agent Example

## Requirements
These are the required _Libraries_ and _Environment Variables_ for this notebook


### Libraries Required


In [ ]:
# for setting up Jupyter widgets and notebook features
%conda install conda-forge::ipywidgets --update-deps --force-reinstall
%conda install conda-forge::ipykernel --update-deps --force-reinstall


- [LangSmith](https://docs.langchain.com/langsmith/home)
- [LangChain](https://reference.langchain.com/python/langchain/langchain/)
- [LangChain Core](https://reference.langchain.com/python/langchain_core/)
- [LangChain Groq Model Provider](https://reference.langchain.com/python/integrations/langchain_groq/)

In [ ]:
%conda install conda-forge::langsmith
%conda install conda-forge::langchain --update-deps --force-reinstall
%conda install conda-forge::langchain-core --update-deps --force-reinstall

%conda install conda-forge::langchain-groq


### Variables Required

| Token Name             | `.env` Name          | Where to Get / Setting Value                                                      |                                                                                              Reference |
| :--------------------- | :------------------- | :-------------------------------------------------------------------------------- | -----------------------------------------------------------------------------------------------------: |
| Groq API Key           | `GROQ_API_KEY`       | [Groq Console](https://console.groq.com/keys)                                     |                                              [Groq API Docs](https://console.groq.com/docs/quickstart) |
| LangSmith API Key      | `LANGSMITH_API_KEY`  | [LangSmith Settings](https://smith.langchain.com/settings)                        |                                   [LangSmith API Reference](https://docs.langchain.com/langsmith/home) |
| LangSmith Tracing      | `LANGSMITH_TRACING`  | A Boolean, set the value to `true` or `false` to enable or disable logging traces | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Endpoint     | `LANGSMITH_ENDPOINT` | The LangSmith Endpoint to log the traces (<https://api.smith.langchain.com>)      | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |
| LangSmith Project Name | `LANGSMITH_PROJECT`  | The name of the project to log the traces under                                   | [LangSmith Observability API Reference](https://docs.langchain.com/langsmith/observability-quickstart) |


In [ ]:
from pathlib import Path
import sys

ROOT = Path().resolve().parent.parent
sys.path.append(str(ROOT))


In [ ]:
from utils import (
    set_env_variables,
)

ENV_FILE = ROOT / ".env"

set_env_variables(env_file=ENV_FILE)


## Actual Shenanigans


In [1]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage


### Defining Tools


<div class="alert alert-success">

**💡 Tip!**

Read the docs [here](https://reference.langchain.com/python/langchain/tools/)

</div>




In [2]:
@tool
def search_arxiv_by_author(author_name: str) -> str:
    """
    Search for recent papers by a specific author on ArXiv.
    This is a mock implementation - in production, integrate with ArXiv API.
    """
    # Mock implementation
    papers_db = {
        "Smith": [
            "Smith et al. (2025) - 'Advances in Climate Modeling' - ArXiv:2501.12345",
            "Smith, J. (2024) - 'Deep Learning for Environmental Data' - ArXiv:2412.54321",
            "Smith et al. (2024) - 'Neural Networks for Climate Prediction' - ArXiv:2411.11111",
        ],
        "Johnson": [
            "Johnson, K. (2025) - 'Machine Learning in Climate Science' - ArXiv:2502.99999",
            "Johnson et al. (2024) - 'Data Analysis Methods for Climate' - ArXiv:2412.77777",
        ],
        "Brown": [
            "Brown, L. (2025) - 'Climate Change Mitigation Strategies' - ArXiv:2501.55555"
        ],
    }

    author = author_name.title()
    if author in papers_db:
        papers = papers_db[author]
        return f"Found {len(papers)} papers by {author}:\n" + "\n".join(
            f"- {p}" for p in papers
        )
    else:
        return f"No papers found for author '{author_name}'. Try searching for: Smith, Johnson, or Brown"


@tool
def search_collaborators(author_name: str) -> str:
    """
    Find collaborators of a specific researcher.
    """
    collaborators_db = {
        "Smith": ["Johnson", "Williams", "Chen"],
        "Johnson": ["Smith", "Brown", "Martinez"],
        "Brown": ["Johnson", "Davis"],
        "Williams": ["Smith", "Garcia"],
        "Chen": ["Smith", "Anderson"],
    }

    author = author_name.title()
    if author in collaborators_db:
        collabs = collaborators_db[author]
        return f"Collaborators of {author}: {', '.join(collabs)}"
    else:
        return f"No collaborator information found for '{author_name}'"


@tool
def search_papers_by_keyword(keyword: str) -> str:
    """
    Search for papers related to a specific keyword in climate science.
    """
    keyword_lower = keyword.lower()
    papers_db = {
        "climate change": [
            "Smith et al. (2025) - 'Advances in Climate Modeling' - ArXiv:2501.12345",
            "Brown, L. (2025) - 'Climate Change Mitigation Strategies' - ArXiv:2501.55555",
            "Johnson, K. (2025) - 'Machine Learning in Climate Science' - ArXiv:2502.99999",
        ],
        "neural networks": [
            "Smith et al. (2024) - 'Neural Networks for Climate Prediction' - ArXiv:2411.11111",
            "Johnson et al. (2024) - 'Data Analysis Methods for Climate' - ArXiv:2412.77777",
        ],
        "deep learning": [
            "Smith, J. (2024) - 'Deep Learning for Environmental Data' - ArXiv:2412.54321"
        ],
    }

    results = []
    for key, papers in papers_db.items():
        if keyword_lower in key:
            results.extend(papers)

    if results:
        return f"Found {len(results)} papers about '{keyword}':\n" + "\n".join(
            f"- {p}" for p in results
        )
    else:
        return f"No papers found for keyword '{keyword}'. Try: climate change, neural networks, or deep learning"


### Initializing the Model and the Agent

In [4]:
def create_research_agent():
    """
    Create a ReAct agent that can research papers and collaborators.
    """
    groq_model = ChatGroq(
        model="llama-3.3-70b-versatile",
        temperature=0.7,
        max_tokens=1024,
    )

    tools = [search_arxiv_by_author, search_collaborators, search_papers_by_keyword]

    system_prompt = """You are a research assistant specialized in climate science and environmental research.
Your task is to help find recent papers and identify collaborators of researchers.

When asked about papers by a researcher's collaborators:
1. First find who collaborates with that researcher
2. Then search for papers by each collaborator
3. Compile and present the results clearly

Be systematic and thorough in your research."""

    agent = create_agent(
        model=groq_model,
        tools=tools,
        system_prompt=system_prompt,
    )
    return agent


In [5]:
def run_agent_query(agent, query: str):
    """
    Execute a query through the agent and display results.
    """
    print(f"\n{'=' * 70}")
    print(f"Query: {query}")
    print(f"{'=' * 70}\n")

    try:
        # Invoke the agent with the user query
        result = agent.invoke({"messages": [HumanMessage(content=query)]})

        # Extract and display the final response
        final_message = result["messages"][-1]
        print("Agent Response:")
        print("-" * 70)
        print(final_message.content)
        print("-" * 70)

        return result

    except Exception as e:
        print(f"Error executing query: {e}")
        import traceback

        traceback.print_exc()


### Running the Agent

#### Demo the agent with example queries

[LangSmith trace](https://smith.langchain.com/public/de99cdc2-e385-476c-b24b-e27180d82b4c/r) for prompt: `Tell me about the researchers collaborating with Johnson on climate research.`

In [7]:
print("=" * 70)
print("LangChain ReAct Agent with LangSmith Tracing")
print("=" * 70)

agent = create_research_agent()

queries = [
    "Find recent papers on climate change by Dr. Smith's collaborators.",
    "What are the latest papers about neural networks in climate science?",
    "Tell me about the researchers collaborating with Johnson on climate research.",
]

for query in queries:
    run_agent_query(agent, query)
    print("\n")


LangChain ReAct Agent with LangSmith Tracing

Query: Find recent papers on climate change by Dr. Smith's collaborators.

Agent Response:
----------------------------------------------------------------------
Based on the search results, Dr. Smith's collaborators could not be found, and therefore, recent papers by their collaborators could not be retrieved. However, some recent papers related to climate change are:

1. 'Advances in Climate Modeling' by Smith et al. (2025) - ArXiv:2501.12345
2. 'Climate Change Mitigation Strategies' by Brown, L. (2025) - ArXiv:2501.55555
3. 'Machine Learning in Climate Science' by Johnson, K. (2025) - ArXiv:2502.99999

These papers may be of interest to those looking for recent research on climate change.
----------------------------------------------------------------------



Query: What are the latest papers about neural networks in climate science?

Agent Response:
----------------------------------------------------------------------
Here are the re

#### Interactive Mode

- [LangSmith Trace](https://smith.langchain.com/public/d2bdb8ad-3a97-4207-9afe-a80edfe0f1d6/r) for prompt: `find recent papers on climate change`
- [LangSmith Trace](https://smith.langchain.com/public/c3a44a73-80fd-424a-8a65-573587472649/r) for prompt: `what about neural networks?`

In [8]:
import ipywidgets as widgets
from IPython.display import display

text = widgets.Text(placeholder="Ask away!")
send = widgets.Button(description="Send")
out = widgets.Output()

print("\n" + "=" * 70)
print("EXAMPLE 3: Interactive Mode")
print("=" * 70)


def on_click(b):
    query = text.value.strip()
    if not query:
        return
    text.value = ""  # clear input
    with out:
        try:
            run_agent_query(agent, query)
        except Exception as e:
            print("Error:", e)


send.on_click(on_click)
display(widgets.VBox([text, send, out]))



EXAMPLE 3: Interactive Mode
